test modules

In [1]:
%run homework_modules.ipynb

--- GlobalMaxPool2d Test ---
Input shape: (2, 3, 4, 5)
Output shape: (2, 3), Expected shape: (2, 3)
Output matches expected: True
Gradient input shape: (2, 3, 4, 5)
All zero-masked grads are zero: True
Number of non-zero grads: 6

--- GlobalAvgPool2d Test ---
Input shape: (2, 3, 4, 5)
Output shape: (2, 3), Expected shape: (2, 3)
Output matches expected: True
Gradient input shape: (2, 3, 4, 5)
Gradient input matches expected: True


In [2]:
import torch
from torch.autograd import Variable
import unittest

In [3]:
class TestLayers(unittest.TestCase):
    def assertNumpyClose(self, a, b, atol=1e-6, rtol=1e-5, msg=''):
        """ Вспомогательный метод для сравнения NumPy массивов с сообщениями об ошибке """
        self.assertTrue(np.allclose(a, b, atol=atol, rtol=rtol),
                        msg=f"{msg}\nExpected:\n{b}\nGot:\n{a}\nDifference:\n{a-b}\nMax Diff: {np.max(np.abs(a-b))}")

    def test_Linear(self):
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in, n_out = 2, 3, 4
        for _ in range(100):
            # Инициализация слоев
            torch_layer = torch.nn.Linear(n_in, n_out)
            custom_layer = Linear(n_in, n_out)
            # Копируем веса из PyTorch для точного сравнения
            custom_layer.W = torch_layer.weight.data.numpy().copy()
            custom_layer.b = torch_layer.bias.data.numpy().copy()

            layer_input = np.random.uniform(-10, 10, (batch_size, n_in)).astype(np.float32)
            next_layer_grad = np.random.uniform(-10, 10, (batch_size, n_out)).astype(np.float32)

            # 1. Проверка выхода слоя
            custom_layer_output = custom_layer.forward(layer_input)
            layer_input_var = Variable(torch.from_numpy(layer_input), requires_grad=True)
            torch_layer_output_var = torch_layer(layer_input_var)
            self.assertNumpyClose(custom_layer_output, torch_layer_output_var.data.numpy(), msg="Linear forward")

            # 2. Проверка градиента по входу
            custom_layer.zeroGradParameters() # Обнуляем градиенты перед backward
            custom_layer_grad = custom_layer.backward(layer_input, next_layer_grad)
            torch_layer_output_var.backward(torch.from_numpy(next_layer_grad))
            torch_layer_grad_var = layer_input_var.grad
            self.assertNumpyClose(custom_layer_grad, torch_layer_grad_var.data.numpy(), msg="Linear gradInput")

            # 3. Проверка градиентов по параметрам
            # Градиенты должны были накопиться в custom_layer.gradW и custom_layer.gradb во время backward
            weight_grad = custom_layer.gradW
            bias_grad = custom_layer.gradb
            torch_weight_grad = torch_layer.weight.grad.data.numpy()
            torch_bias_grad = torch_layer.bias.grad.data.numpy()
            self.assertNumpyClose(weight_grad, torch_weight_grad, msg="Linear gradW")
            self.assertNumpyClose(bias_grad, torch_bias_grad, msg="Linear gradb")

    def test_SoftMax(self):
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in = 2, 4
        for _ in range(100):
            torch_layer = torch.nn.Softmax(dim=1)
            custom_layer = SoftMax()

            layer_input = np.random.uniform(-10, 10, (batch_size, n_in)).astype(np.float32)
            # gradOutput должен иметь ту же форму, что и output
            next_layer_grad = np.random.uniform(-1, 1, (batch_size, n_in)).astype(np.float32)

            # 1. Проверка выхода
            custom_layer_output = custom_layer.forward(layer_input)
            layer_input_var = Variable(torch.from_numpy(layer_input), requires_grad=True)
            torch_layer_output_var = torch_layer(layer_input_var)
            self.assertNumpyClose(custom_layer_output, torch_layer_output_var.data.numpy(), atol=1e-7, msg="SoftMax forward") # Увеличил точность

            # 2. Проверка градиента по входу
            custom_layer_grad = custom_layer.backward(layer_input, next_layer_grad)
            torch_layer_output_var.backward(torch.from_numpy(next_layer_grad))
            torch_layer_grad_var = layer_input_var.grad
            self.assertNumpyClose(custom_layer_grad, torch_layer_grad_var.data.numpy(), atol=1e-7, msg="SoftMax gradInput") # Увеличил точность

    def test_LogSoftMax(self):
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in = 2, 4
        for _ in range(100):
            torch_layer = torch.nn.LogSoftmax(dim=1)
            custom_layer = LogSoftMax()

            layer_input = np.random.uniform(-10, 10, (batch_size, n_in)).astype(np.float32)
            next_layer_grad = np.random.uniform(-1, 1, (batch_size, n_in)).astype(np.float32)

            # 1. Проверка выхода
            custom_layer_output = custom_layer.forward(layer_input)
            layer_input_var = Variable(torch.from_numpy(layer_input), requires_grad=True)
            torch_layer_output_var = torch_layer(layer_input_var)
            self.assertNumpyClose(custom_layer_output, torch_layer_output_var.data.numpy(), msg="LogSoftMax forward")

            # 2. Проверка градиента по входу
            custom_layer_grad = custom_layer.backward(layer_input, next_layer_grad)
            torch_layer_output_var.backward(torch.from_numpy(next_layer_grad))
            torch_layer_grad_var = layer_input_var.grad
            self.assertNumpyClose(custom_layer_grad, torch_layer_grad_var.data.numpy(), msg="LogSoftMax gradInput")


    def test_BatchNormalization(self):
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in = 32, 16
        # Тест для 2D входа
        for _ in range(100):
            # Используем momentum напрямую, как в классе
            momentum = 0.1 # Стандартное значение momentum
            eps = BatchNormalization.EPS

            # --- Режим обучения ---
            # Инициализируем наш слой с momentum
            custom_layer = BatchNormalization(n_in, momentum=momentum)
            custom_layer.train()
            # PyTorch слой с тем же momentum
            torch_layer = torch.nn.BatchNorm1d(n_in, eps=eps, momentum=momentum, affine=False) # affine=False, т.к. тестируем только нормализацию
            torch_layer.train()
            # Синхронизируем начальные moving_mean и moving_variance
            custom_layer.moving_mean = torch_layer.running_mean.numpy().copy()
            custom_layer.moving_variance = torch_layer.running_var.numpy().copy()

            layer_input = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)
            next_layer_grad = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)

            # 1. Проверка выхода (train)
            custom_layer_output = custom_layer.forward(layer_input)
            layer_input_var = Variable(torch.from_numpy(layer_input), requires_grad=True)
            torch_layer_output_var = torch_layer(layer_input_var)
            self.assertNumpyClose(custom_layer_output, torch_layer_output_var.data.numpy(), atol=1e-6, msg="BN forward (train)")

            # 2. Проверка градиента по входу (train)
            custom_layer_grad = custom_layer.backward(layer_input, next_layer_grad)
            torch_layer_output_var.backward(torch.from_numpy(next_layer_grad))
            torch_layer_grad_var = layer_input_var.grad
            self.assertNumpyClose(custom_layer_grad, torch_layer_grad_var.data.numpy(), atol=1e-5, msg="BN gradInput (train)")

            # 3. Проверка скользящих средних (после forward/backward)
            self.assertNumpyClose(custom_layer.moving_mean, torch_layer.running_mean.numpy(), atol=1e-6, msg="BN moving_mean")
            # Дисперсию не сравниваем напрямую из-за разницы biased/unbiased в обновлении
            # self.assertNumpyClose(custom_layer.moving_variance, torch_layer.running_var.numpy(), atol=1e-5, msg="BN moving_variance")


            # --- Режим оценки ---
            # Важно: используем те же скользящие статистики, которые были обновлены в режиме train
            custom_layer.evaluate()
            torch_layer.eval()

            # Используем новые данные для оценки
            eval_input = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)
            eval_next_layer_grad = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)

            # 4. Проверка выхода (eval)
            custom_layer_output_eval = custom_layer.forward(eval_input)
            eval_input_var = Variable(torch.from_numpy(eval_input), requires_grad=True)
            # Убедимся, что у PyTorch слоя requires_grad=False для параметров в eval режиме,
            # но для входа grad все еще нужен для теста backward
            torch_layer_output_eval_var = torch_layer(eval_input_var)
            self.assertNumpyClose(custom_layer_output_eval, torch_layer_output_eval_var.data.numpy(), atol=1e-6, msg="BN forward (eval)")

            # 5. Проверка градиента по входу (eval)
            custom_layer_grad_eval = custom_layer.backward(eval_input, eval_next_layer_grad)
            # Обнуляем градиент входа перед backward в режиме eval
            if eval_input_var.grad is not None:
                eval_input_var.grad.zero_()
            torch_layer_output_eval_var.backward(torch.from_numpy(eval_next_layer_grad))
            torch_layer_grad_eval_var = eval_input_var.grad
            self.assertNumpyClose(custom_layer_grad_eval, torch_layer_grad_eval_var.data.numpy(), atol=1e-6, msg="BN gradInput (eval)")


    def test_Sequential_BatchNormAffine(self):
        # Тестируем Sequential на примере BatchNorm1d(affine=True)
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in = 32, 16
        for _ in range(100):
            # Используем momentum как в PyTorch
            momentum = 0.1
            eps = BatchNormalization.EPS

            # --- Режим обучения ---
            # PyTorch слой
            torch_layer = torch.nn.BatchNorm1d(n_in, eps=eps, momentum=momentum, affine=True)
            torch_layer.train()
            # Инициализируем веса и смещения в torch случайными значениями
            torch_layer.weight.data = torch.from_numpy(np.random.rand(n_in).astype(np.float32))
            torch_layer.bias.data = torch.from_numpy(np.random.rand(n_in).astype(np.float32))

            # Наш Sequential эквивалент
            custom_layer = Sequential()
            # Передаем momentum в наш BN
            bn_layer = BatchNormalization(n_in, momentum=momentum)
            scaling_layer = ChannelwiseScaling(n_in)

            # Синхронизируем параметры и статистики
            bn_layer.moving_mean = torch_layer.running_mean.numpy().copy()
            bn_layer.moving_variance = torch_layer.running_var.numpy().copy()
            scaling_layer.gamma = torch_layer.weight.data.numpy().copy()
            scaling_layer.beta = torch_layer.bias.data.numpy().copy()

            custom_layer.add(bn_layer)
            custom_layer.add(scaling_layer)
            custom_layer.train() # Устанавливаем режим train для Sequential и всех его слоев

            layer_input = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)
            next_layer_grad = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)

            # 1. Проверка выхода (train)
            custom_layer_output = custom_layer.forward(layer_input)
            layer_input_var = Variable(torch.from_numpy(layer_input), requires_grad=True)
            torch_layer_output_var = torch_layer(layer_input_var)
            self.assertNumpyClose(custom_layer_output, torch_layer_output_var.data.numpy(), atol=1e-6, msg="Sequential BN forward (train)")

            # 2. Проверка градиента по входу (train)
            custom_layer.zeroGradParameters() # Обнуляем градиенты перед backward
            custom_layer_grad = custom_layer.backward(layer_input, next_layer_grad)
            torch_layer_output_var.backward(torch.from_numpy(next_layer_grad))
            torch_layer_grad_var = layer_input_var.grad
            # Может потребоваться чуть большая погрешность из-за двух слоев
            self.assertNumpyClose(custom_layer_grad, torch_layer_grad_var.data.numpy(), atol=1e-5, rtol=1e-4, msg="Sequential BN gradInput (train)")

            # 3. Проверка градиентов по параметрам (gamma, beta)
            grad_params = custom_layer.getGradParameters()
            self.assertEqual(len(grad_params), 2)
            weight_grad = grad_params[0] # gradGamma
            bias_grad = grad_params[1]   # gradBeta

            torch_weight_grad = torch_layer.weight.grad.data.numpy()
            torch_bias_grad = torch_layer.bias.grad.data.numpy()

            # Увеличиваем atol для градиентов параметров из-за возможного накопления ошибок
            self.assertNumpyClose(weight_grad, torch_weight_grad, atol=1e-5, rtol=1e-4, msg="Sequential BN gradGamma (train)")
            self.assertNumpyClose(bias_grad, torch_bias_grad, atol=1e-5, rtol=1e-4, msg="Sequential BN gradBeta (train)")

            # --- Режим оценки ---
            custom_layer.evaluate()
            torch_layer.eval()

            eval_input = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)
            eval_next_grad = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)

            # 4. Проверка выхода (eval)
            custom_layer_output_eval = custom_layer.forward(eval_input)
            eval_input_var = Variable(torch.from_numpy(eval_input), requires_grad=True)
            torch_layer_output_eval_var = torch_layer(eval_input_var)
            self.assertNumpyClose(custom_layer_output_eval, torch_layer_output_eval_var.data.numpy(), atol=1e-6, msg="Sequential BN forward (eval)")

            # 5. Проверка градиента по входу (eval)
            custom_layer_grad_eval = custom_layer.backward(eval_input, eval_next_grad)
            # Нужно вызвать backward и для PyTorch слоя в eval режиме
            # Обнуляем градиенты перед новым backward
            if eval_input_var.grad is not None:
                eval_input_var.grad.zero_()
            if torch_layer.weight.grad is not None:
                torch_layer.weight.grad.zero_()
            if torch_layer.bias.grad is not None:
                torch_layer.bias.grad.zero_()

            torch_layer_output_eval_var.backward(torch.from_numpy(eval_next_grad))
            torch_layer_grad_eval_var = eval_input_var.grad
            self.assertNumpyClose(custom_layer_grad_eval, torch_layer_grad_eval_var.data.numpy(), atol=1e-6, msg="Sequential BN gradInput (eval)")
    


    def test_Dropout(self):
        np.random.seed(42)

        batch_size, n_in = 100, 50 # Больший размер для статистических проверок
        for p in [0.0, 0.3, 0.5, 0.8]:
          with self.subTest(p=p):
            layer = Dropout(p)

            layer_input = np.random.uniform(1, 5, (batch_size, n_in)).astype(np.float32) # Положительные входы
            next_layer_grad = np.random.uniform(1, 5, (batch_size, n_in)).astype(np.float32) # Положительные градиенты

            # --- Режим обучения ---
            layer.train()
            layer_output = layer.forward(layer_input)
            layer_grad = layer.backward(layer_input, next_layer_grad)

            # 1. Проверка выхода (train)
            # Элементы должны быть либо 0, либо input * scale
            scale = 1.0 / (1.0 - p) if p < 1.0 else 0.0
            is_zero_mask_out = np.isclose(layer_output, 0)
            is_scaled_mask_out = np.isclose(layer_output, layer_input * scale)
            # Каждый элемент должен быть либо нулем, либо масштабированным
            self.assertTrue(np.all(np.logical_or(is_zero_mask_out, is_scaled_mask_out)), msg=f"Dropout forward (train, p={p}) output values")

            # Проверяем долю нулей (статистически)
            zero_fraction = np.mean(is_zero_mask_out)
            # Ожидаемая доля нулей - p
            # Используем большую погрешность из-за случайности
            self.assertAlmostEqual(zero_fraction, p, delta=0.05, msg=f"Dropout forward (train, p={p}) zero fraction")

            # 2. Проверка градиента по входу (train)
            # Градиент должен быть либо 0, либо next_layer_grad * scale
            is_zero_mask_grad = np.isclose(layer_grad, 0)
            is_scaled_mask_grad = np.isclose(layer_grad, next_layer_grad * scale)
            self.assertTrue(np.all(np.logical_or(is_zero_mask_grad, is_scaled_mask_grad)), msg=f"Dropout backward (train, p={p}) grad values")

            # Маски нулей для выхода и градиента должны совпадать
            self.assertTrue(np.all(is_zero_mask_out == is_zero_mask_grad), msg=f"Dropout masks match (train, p={p})")


            # --- Режим оценки ---
            layer.evaluate()
            eval_input = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)
            eval_next_grad = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)

            # 3. Проверка выхода (eval)
            eval_output = layer.forward(eval_input)
            self.assertNumpyClose(eval_output, eval_input, msg=f"Dropout forward (eval, p={p})")

            # 4. Проверка градиента по входу (eval)
            eval_grad = layer.backward(eval_input, eval_next_grad)
            self.assertNumpyClose(eval_grad, eval_next_grad, msg=f"Dropout backward (eval, p={p})")


    def test_Conv2d(self):
        hyperparams = [
            {'batch_size': 2, 'in_channels': 3, 'out_channels': 4, 'height': 5, 'width': 6,
             'kernel_size': 3, 'stride': 1, 'padding': 1, 'bias': True},
            {'batch_size': 4, 'in_channels': 1, 'out_channels': 2, 'height': 7, 'width': 7,
             'kernel_size': (3,3), 'stride': 2, 'padding': (1,1), 'bias': False},
            {'batch_size': 2, 'in_channels': 2, 'out_channels': 3, 'height': 8, 'width': 6,
             'kernel_size': (2,3), 'stride': (1,2), 'padding': (0,1), 'bias': True},
             {'batch_size': 1, 'in_channels': 1, 'out_channels': 1, 'height': 3, 'width': 3,
             'kernel_size': 2, 'stride': 1, 'padding': 0, 'bias': False},
        ]
        np.random.seed(42)
        torch.manual_seed(42)

        for params in hyperparams:
              with self.subTest(params=params):
                  batch_size = params['batch_size']
                  in_channels = params['in_channels']
                  out_channels = params['out_channels']
                  height = params['height']
                  width = params['width']
                  kernel_size = params['kernel_size']
                  stride = params['stride']
                  padding = params['padding']
                  bias = params['bias']
                  # Наша реализация пока поддерживает только padding_mode='zeros'
                  padding_mode = 'zeros'

                  custom_layer = Conv2d(in_channels, out_channels, kernel_size,
                                        stride=stride, padding=padding, bias=bias,
                                        padding_mode=padding_mode)
                  custom_layer.train()

                  torch_layer = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                                stride=stride, padding=padding, bias=bias,
                                                padding_mode=padding_mode)
                  torch_layer.train()

                  # Копируем веса
                  custom_layer.W = torch_layer.weight.detach().numpy().copy()
                  if bias:
                      custom_layer.b = torch_layer.bias.detach().numpy().copy()

                  layer_input = np.random.randn(batch_size, in_channels, height, width).astype(np.float32)
                  input_var = torch.tensor(layer_input, requires_grad=True)

                  # 1. Проверка выхода
                  custom_output = custom_layer.forward(layer_input)
                  torch_output = torch_layer(input_var)
                  self.assertNumpyClose(custom_output, torch_output.detach().numpy(), atol=1e-6, msg=f"Conv2d forward {params}")

                  # 2. Проверка градиентов (вход и параметры)
                  next_layer_grad = np.random.randn(*custom_output.shape).astype(np.float32)
                  custom_layer.zeroGradParameters()
                  custom_grad_input = custom_layer.backward(layer_input, next_layer_grad)
                  torch_output.backward(torch.tensor(next_layer_grad))
                  torch_grad_input = input_var.grad.detach().numpy()

                  # Проверка градиента по входу
                  self.assertNumpyClose(custom_grad_input, torch_grad_input, atol=1e-5, msg=f"Conv2d gradInput {params}")

                  # Проверка градиента по весам (W)
                  custom_gradW = custom_layer.gradW
                  torch_gradW = torch_layer.weight.grad.detach().numpy()
                  self.assertNumpyClose(custom_gradW, torch_gradW, atol=1e-5, msg=f"Conv2d gradW {params}")

                  # Проверка градиента по смещению (b), если есть
                  if bias:
                      custom_gradb = custom_layer.gradb
                      torch_gradb = torch_layer.bias.grad.detach().numpy()
                      self.assertNumpyClose(custom_gradb, torch_gradb, atol=1e-5, msg=f"Conv2d gradb {params}")


    def test_LeakyReLU(self):
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in = 2, 4
        for _ in range(100):
            slope = np.random.uniform(0.01, 0.2) # Разные значения slope
            torch_layer = torch.nn.LeakyReLU(slope)
            custom_layer = LeakyReLU(slope)

            layer_input = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)
            next_layer_grad = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)

            # 1. Проверка выхода
            custom_layer_output = custom_layer.forward(layer_input)
            layer_input_var = Variable(torch.from_numpy(layer_input), requires_grad=True)
            torch_layer_output_var = torch_layer(layer_input_var)
            self.assertNumpyClose(custom_layer_output, torch_layer_output_var.data.numpy(), msg="LeakyReLU forward")

            # 2. Проверка градиента по входу
            custom_layer_grad = custom_layer.backward(layer_input, next_layer_grad)
            torch_layer_output_var.backward(torch.from_numpy(next_layer_grad))
            torch_layer_grad_var = layer_input_var.grad
            self.assertNumpyClose(custom_layer_grad, torch_layer_grad_var.data.numpy(), msg="LeakyReLU gradInput")

    def test_ELU(self):
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in = 2, 4
        for _ in range(100):
            alpha = np.random.uniform(0.5, 1.5) # Разные значения alpha
            torch_layer = torch.nn.ELU(alpha)
            custom_layer = ELU(alpha)

            layer_input = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)
            next_layer_grad = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)

            # 1. Проверка выхода
            custom_layer_output = custom_layer.forward(layer_input)
            layer_input_var = Variable(torch.from_numpy(layer_input), requires_grad=True)
            torch_layer_output_var = torch_layer(layer_input_var)
            self.assertNumpyClose(custom_layer_output, torch_layer_output_var.data.numpy(), msg="ELU forward")

            # 2. Проверка градиента по входу
            custom_layer_grad = custom_layer.backward(layer_input, next_layer_grad)
            torch_layer_output_var.backward(torch.from_numpy(next_layer_grad))
            torch_layer_grad_var = layer_input_var.grad
            self.assertNumpyClose(custom_layer_grad, torch_layer_grad_var.data.numpy(), msg="ELU gradInput")

    def test_SoftPlus(self):
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in = 2, 4
        for _ in range(100):
            torch_layer = torch.nn.Softplus()
            custom_layer = SoftPlus()

            layer_input = np.random.uniform(-10, 10, (batch_size, n_in)).astype(np.float32) # Больший диапазон
            next_layer_grad = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)

            # 1. Проверка выхода
            custom_layer_output = custom_layer.forward(layer_input)
            layer_input_var = Variable(torch.from_numpy(layer_input), requires_grad=True)
            torch_layer_output_var = torch_layer(layer_input_var)
            # Softplus может давать большие значения, увеличим rtol
            self.assertNumpyClose(custom_layer_output, torch_layer_output_var.data.numpy(), rtol=1e-4, msg="SoftPlus forward")

            # 2. Проверка градиента по входу
            custom_layer_grad = custom_layer.backward(layer_input, next_layer_grad)
            torch_layer_output_var.backward(torch.from_numpy(next_layer_grad))
            torch_layer_grad_var = layer_input_var.grad
            self.assertNumpyClose(custom_layer_grad, torch_layer_grad_var.data.numpy(), rtol=1e-4, msg="SoftPlus gradInput")


    def test_ClassNLLCriterionUnstable(self):
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in = 5, 10 # Больше классов
        for _ in range(100):
            # Используем нашу реализацию NLL, сравниваем с PyTorch NLLLoss
            custom_criterion = ClassNLLCriterionUnstable()
            # PyTorch NLLLoss ожидает log-вероятности, поэтому берем log от нашего входа
            torch_criterion = torch.nn.NLLLoss()

            # Генерируем вероятности (выход SoftMax)
            logits = np.random.uniform(-2, 2, (batch_size, n_in)).astype(np.float32)
            probs = SoftMax().forward(logits) # Используем нашу реализацию SoftMax
            # Убедимся, что вероятности корректны
            probs = np.clip(probs, custom_criterion.EPS, 1. - custom_criterion.EPS)
            probs /= probs.sum(axis=-1, keepdims=True) # Нормализуем на всякий случай

            # Генерируем one-hot target
            target_labels = np.random.randint(0, n_in, batch_size)
            target_one_hot = np.zeros((batch_size, n_in), np.float32)
            target_one_hot[np.arange(batch_size), target_labels] = 1.0

            # 1. Проверка значения лосса
            custom_loss = custom_criterion.forward(probs, target_one_hot)
            # Для PyTorch нужен log(probs) и метки классов (не one-hot)
            log_probs_torch = torch.log(torch.from_numpy(probs))
            target_labels_torch = torch.from_numpy(target_labels).long()
            torch_loss = torch_criterion(log_probs_torch, target_labels_torch)
            self.assertAlmostEqual(custom_loss, torch_loss.item(), delta=1e-6, msg="NLLUnstable forward")

            # 2. Проверка градиента по входу (probs)
            custom_grad_input = custom_criterion.backward(probs, target_one_hot)
            # Чтобы получить градиент по probs в PyTorch, нужно сделать requires_grad=True для log_probs
            log_probs_torch_var = Variable(log_probs_torch.data, requires_grad=True)
            log_probs_torch_var.retain_grad() # Нужно для промежуточных переменных
            # dL/d(probs) = dL/d(log_probs) * d(log_probs)/d(probs) = grad(log_probs) / probs
            torch_loss_for_grad = torch_criterion(log_probs_torch_var, target_labels_torch)
            torch_loss_for_grad.backward()
            torch_grad_log_probs = log_probs_torch_var.grad.data.numpy()
            # Численно нестабильно делить на probs, сравним с нашей формулой: -target / (probs * N)
            expected_torch_grad = - target_one_hot / (probs * batch_size)
            # Сравниваем наш градиент с теоретическим градиентом PyTorch
            self.assertNumpyClose(custom_grad_input, expected_torch_grad, atol=1e-6, msg="NLLUnstable gradInput")


    def test_ClassNLLCriterion(self):
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in = 5, 10
        for _ in range(100):
            # Наша реализация NLL (стабильная)
            custom_criterion = ClassNLLCriterion()
            # PyTorch NLLLoss
            torch_criterion = torch.nn.NLLLoss()

            # Генерируем log-вероятности (выход LogSoftMax)
            logits = np.random.uniform(-5, 5, (batch_size, n_in)).astype(np.float32)
            log_probs = LogSoftMax().forward(logits) # Используем нашу реализацию LogSoftMax

            # Генерируем one-hot target
            target_labels = np.random.randint(0, n_in, batch_size)
            target_one_hot = np.zeros((batch_size, n_in), np.float32)
            target_one_hot[np.arange(batch_size), target_labels] = 1.0

            # 1. Проверка значения лосса
            custom_loss = custom_criterion.forward(log_probs, target_one_hot)
            # PyTorch принимает log-вероятности и метки классов
            log_probs_torch = torch.from_numpy(log_probs)
            target_labels_torch = torch.from_numpy(target_labels).long()
            torch_loss = torch_criterion(log_probs_torch, target_labels_torch)
            self.assertAlmostEqual(custom_loss, torch_loss.item(), delta=1e-6, msg="NLLStable forward")

            # 2. Проверка градиента по входу (log_probs)
            custom_grad_input = custom_criterion.backward(log_probs, target_one_hot)
            # PyTorch NLLLoss сразу дает градиент по log_probs
            log_probs_torch_var = Variable(log_probs_torch.data, requires_grad=True)
            torch_loss_for_grad = torch_criterion(log_probs_torch_var, target_labels_torch)
            torch_loss_for_grad.backward()
            torch_grad_input = log_probs_torch_var.grad.data.numpy()
            # Сравниваем градиенты напрямую
            self.assertNumpyClose(custom_grad_input, torch_grad_input, atol=1e-7, msg="NLLStable gradInput") # Повышенная точность


    def test_MaxPool2d(self):
        hyperparams = [
            {'batch_size': 2, 'channels': 3, 'height': 5, 'width': 6, 'kernel_size': 2, 'stride': 2, 'padding': 0},
            {'batch_size': 4, 'channels': 1, 'height': 7, 'width': 7, 'kernel_size': 3, 'stride': 1, 'padding': 1},
            {'batch_size': 2, 'channels': 2, 'height': 8, 'width': 6, 'kernel_size': (2,3), 'stride': (2,1), 'padding': (1,0)},
            {'batch_size': 1, 'channels': 1, 'height': 4, 'width': 4, 'kernel_size': 4, 'stride': 4, 'padding': 0}, # Global-like
        ]
        np.random.seed(42)
        torch.manual_seed(42)

        for params in hyperparams:
          with self.subTest(params=params):
              kernel_size = params['kernel_size']
              stride = params['stride']
              padding = params['padding']

              custom_module = MaxPool2d(kernel_size, stride, padding)
              custom_module.train() # Не влияет, но для консистентности

              # PyTorch MaxPool2d с return_indices=True для сравнения градиентов
              torch_module = torch.nn.MaxPool2d(kernel_size, stride=stride, padding=padding, return_indices=False) # Пока без индексов
              torch_module.train()

              input_np = np.random.randn(params['batch_size'], params['channels'], params['height'], params['width']).astype(np.float32)
              input_var = torch.tensor(input_np, requires_grad=True)

              # 1. Проверка выхода
              custom_output = custom_module.forward(input_np)
              torch_output = torch_module(input_var)
              self.assertNumpyClose(custom_output, torch_output.detach().numpy(), atol=1e-7, msg=f"MaxPool2d forward {params}") # Повышенная точность

              # 2. Проверка градиента по входу
              next_grad = np.random.randn(*custom_output.shape).astype(np.float32)
              custom_grad = custom_module.backward(input_np, next_grad)
              torch_output.backward(torch.tensor(next_grad))
              torch_grad = input_var.grad.detach().numpy()
              # Для MaxPool градиенты могут сильно различаться при одинаковых значениях в окне
              # Снижаем требования к точности
              self.assertNumpyClose(custom_grad, torch_grad, atol=1e-6, rtol=1e-5, msg=f"MaxPool2d gradInput {params}")


    def test_AvgPool2d(self):
        hyperparams = [
            {'batch_size': 2, 'channels': 3, 'height': 5, 'width': 6, 'kernel_size': 2, 'stride': 2, 'padding': 0},
            {'batch_size': 4, 'channels': 1, 'height': 7, 'width': 7, 'kernel_size': 3, 'stride': 1, 'padding': 1},
            {'batch_size': 2, 'channels': 2, 'height': 8, 'width': 6, 'kernel_size': (2,3), 'stride': (2,1), 'padding': (1,0)},
             {'batch_size': 1, 'channels': 1, 'height': 4, 'width': 4, 'kernel_size': 4, 'stride': 4, 'padding': 0}, # Global-like
        ]
        np.random.seed(42)
        torch.manual_seed(42)

        for params in hyperparams:
          with self.subTest(params=params):
              kernel_size = params['kernel_size']
              stride = params['stride']
              padding = params['padding']

              custom_module = AvgPool2d(kernel_size, stride, padding)
              custom_module.train()

              torch_module = torch.nn.AvgPool2d(kernel_size, stride=stride, padding=padding)
              torch_module.train()

              input_np = np.random.randn(params['batch_size'], params['channels'], params['height'], params['width']).astype(np.float32)
              input_var = torch.tensor(input_np, requires_grad=True)

              # 1. Проверка выхода
              custom_output = custom_module.forward(input_np)
              torch_output = torch_module(input_var)
              self.assertNumpyClose(custom_output, torch_output.detach().numpy(), atol=1e-7, msg=f"AvgPool2d forward {params}")

              # 2. Проверка градиента по входу
              next_grad = np.random.randn(*custom_output.shape).astype(np.float32)
              custom_grad = custom_module.backward(input_np, next_grad)
              torch_output.backward(torch.tensor(next_grad))
              torch_grad = input_var.grad.detach().numpy()
              self.assertNumpyClose(custom_grad, torch_grad, atol=1e-6, rtol=1e-5, msg=f"AvgPool2d gradInput {params}")


    def test_GlobalMaxPool2d(self):
        np.random.seed(42)
        torch.manual_seed(42)
        batch_size, channels, height, width = 4, 3, 8, 6 # Разные H, W

        for _ in range(10): # Меньше итераций, т.к. нет параметров
          custom_module = GlobalMaxPool2d()
          # Эквивалент в PyTorch - AdaptiveMaxPool2d с output_size=(1, 1)
          torch_module = torch.nn.AdaptiveMaxPool2d((1, 1))

          input_np = np.random.randn(batch_size, channels, height, width).astype(np.float32)
          input_var = torch.tensor(input_np, requires_grad=True)

          # 1. Проверка выхода
          # Наш модуль возвращает (N, C), PyTorch возвращает (N, C, 1, 1)
          custom_output = custom_module.forward(input_np)
          torch_output = torch_module(input_var)
          torch_output_squeezed = torch_output.squeeze(dim=(2, 3)) # Убираем оси (1, 1)
          self.assertNumpyClose(custom_output, torch_output_squeezed.detach().numpy(), atol=1e-7, msg="GlobalMaxPool2d forward")

          # 2. Проверка градиента по входу
          # gradOutput для нашего модуля - (N, C)
          next_grad_custom = np.random.randn(*custom_output.shape).astype(np.float32)
          # gradOutput для PyTorch - (N, C, 1, 1)
          next_grad_torch = torch.tensor(next_grad_custom.reshape(batch_size, channels, 1, 1))

          custom_grad = custom_module.backward(input_np, next_grad_custom)
          torch_output.backward(next_grad_torch)
          torch_grad = input_var.grad.detach().numpy()
          # Снижаем требования к точности для MaxPool
          self.assertNumpyClose(custom_grad, torch_grad, atol=1e-6, rtol=1e-5, msg="GlobalMaxPool2d gradInput")


    def test_GlobalAvgPool2d(self):
        np.random.seed(42)
        torch.manual_seed(42)
        batch_size, channels, height, width = 4, 3, 8, 6 # Разные H, W

        for _ in range(10):
          custom_module = GlobalAvgPool2d()
          torch_module = torch.nn.AdaptiveAvgPool2d((1, 1))

          input_np = np.random.randn(batch_size, channels, height, width).astype(np.float32)
          input_var = torch.tensor(input_np, requires_grad=True)

          # 1. Проверка выхода
          custom_output = custom_module.forward(input_np) # (N, C)
          torch_output = torch_module(input_var) # (N, C, 1, 1)
          torch_output_squeezed = torch_output.squeeze(dim=(2, 3))
          self.assertNumpyClose(custom_output, torch_output_squeezed.detach().numpy(), atol=1e-7, msg="GlobalAvgPool2d forward")

          # 2. Проверка градиента по входу
          next_grad_custom = np.random.randn(*custom_output.shape).astype(np.float32) # (N, C)
          next_grad_torch = torch.tensor(next_grad_custom.reshape(batch_size, channels, 1, 1)) # (N, C, 1, 1)

          custom_grad = custom_module.backward(input_np, next_grad_custom)
          torch_output.backward(next_grad_torch)
          torch_grad = input_var.grad.detach().numpy()
          self.assertNumpyClose(custom_grad, torch_grad, atol=1e-6, rtol=1e-5, msg="GlobalAvgPool2d gradInput")


    def test_Flatten(self):
        np.random.seed(42)
        torch.manual_seed(42)

        # Тестируем только стандартный Flatten (с start_dim=1)
        custom_module = Flatten()
        # PyTorch Flatten по умолчанию тоже использует start_dim=1
        torch_module = torch.nn.Flatten()

        # Разные формы входа
        input_shapes = [(2, 3, 4, 5), (10, 1), (5, 2, 1, 3)]

        for shape in input_shapes:
          with self.subTest(shape=shape):
              input_np = np.random.randn(*shape).astype(np.float32)
              input_var = torch.tensor(input_np, requires_grad=True)

              # 1. Проверка выхода
              custom_output = custom_module.forward(input_np)
              torch_output = torch_module(input_var)
              self.assertNumpyClose(custom_output, torch_output.detach().numpy(), msg=f"Flatten forward {shape}")

              # 2. Проверка градиента по входу
              next_grad = np.random.randn(*custom_output.shape).astype(np.float32)
              custom_grad = custom_module.backward(input_np, next_grad)
              torch_output.backward(torch.tensor(next_grad))
              torch_grad = input_var.grad.detach().numpy()
              self.assertNumpyClose(custom_grad, torch_grad, msg=f"Flatten gradInput {shape}")


    def test_Gelu(self):
        np.random.seed(42)
        torch.manual_seed(42)

        batch_size, n_in = 10, 5
        for _ in range(100):
          custom_module = Gelu()
          custom_module.train() # Не влияет, но для консистентности

          torch_module = torch.nn.GELU(approximate='none')
          torch_module.train()

          input_np = np.random.randn(batch_size, n_in).astype(np.float32)
          input_var = torch.tensor(input_np, requires_grad=True)

          # 1. Проверка выхода
          custom_output = custom_module.forward(input_np)
          torch_output = torch_module(input_var)
          # Увеличиваем atol до 1e-6 из-за возможных различий в точности erf
          self.assertNumpyClose(custom_output, torch_output.detach().numpy(), atol=1e-6, msg="Gelu forward")

          # 2. Проверка градиента по входу
          next_grad = np.random.randn(*custom_output.shape).astype(np.float32)
          custom_grad = custom_module.backward(input_np, next_grad)
          torch_output.backward(torch.tensor(next_grad))
          torch_grad = input_var.grad.detach().numpy()
          self.assertNumpyClose(custom_grad, torch_grad, atol=1e-6, rtol=1e-5, msg="Gelu gradInput")


# Запуск тестов
if __name__ == '__main__':
    # unittest.main(argv=['first-arg-is-ignored'], exit=False)
    # Запускаем тесты и выводим результат
    suite = unittest.TestLoader().loadTestsFromTestCase(TestLayers)
    runner = unittest.TextTestRunner(verbosity=2) # verbosity=2 для более детального вывода
    result = runner.run(suite)

    # Печатаем итоги
    print("\n--- Test Summary ---")
    print(f"Ran: {result.testsRun} tests")
    if result.wasSuccessful():
        print("Result: OK")
    else:
        print("Result: FAILED")
        print(f"Errors: {len(result.errors)}")
        for test, err in result.errors:
            print(f"\nERROR in {test}:\n{err}")
        print(f"Failures: {len(result.failures)}")
        for test, fail in result.failures:
            print(f"\nFAILURE in {test}:\n{fail}")

test_AvgPool2d (__main__.TestLayers.test_AvgPool2d) ... ok
test_BatchNormalization (__main__.TestLayers.test_BatchNormalization) ... ok
test_ClassNLLCriterion (__main__.TestLayers.test_ClassNLLCriterion) ... ok
test_ClassNLLCriterionUnstable (__main__.TestLayers.test_ClassNLLCriterionUnstable) ... ok
test_Conv2d (__main__.TestLayers.test_Conv2d) ... ok
test_Dropout (__main__.TestLayers.test_Dropout) ... ok
test_ELU (__main__.TestLayers.test_ELU) ... ok
test_Flatten (__main__.TestLayers.test_Flatten) ... ok
test_Gelu (__main__.TestLayers.test_Gelu) ... ok
test_GlobalAvgPool2d (__main__.TestLayers.test_GlobalAvgPool2d) ... ok
test_GlobalMaxPool2d (__main__.TestLayers.test_GlobalMaxPool2d) ... ok
test_LeakyReLU (__main__.TestLayers.test_LeakyReLU) ... ok
test_Linear (__main__.TestLayers.test_Linear) ... ok
test_LogSoftMax (__main__.TestLayers.test_LogSoftMax) ... ok
test_MaxPool2d (__main__.TestLayers.test_MaxPool2d) ... ok
test_Sequential_BatchNormAffine (__main__.TestLayers.test_Sequent


--- Test Summary ---
Ran: 18 tests
Result: OK
